In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from textblob import Word
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split,cross_val_score, KFold

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,classification_report
from sklearn.metrics import recall_score,precision_score,precision_recall_fscore_support
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package ave

True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
train_data = '/content/gdrive/MyDrive/NLP_Project/labels.csv'
test_data = '/content/gdrive/MyDrive/NLP_Project/2000_testdata.csv'



In [ ]:
#basic preprocessing

#removing punctuation
def remove_punct_numbers(words):
    return [word.lower() for word in words if word.isalpha()]

# remove stop words
stop_words = nltk.corpus.stopwords.words('english')
def remove_stopwords(words):
    return [word for word in words if word not in stop_words]


# lemmatize
lematizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def lematize(words):
    words = [lematizer.lemmatize(word,pos=get_wordnet_pos(word)) for word in words]
    #words = [lematizer.lemmatize(word,pos='v') for word in words]
    return words


In [ ]:
#Preprocessing train data and test data 

def input_func(file, label):
    #input_path = input('/content/labels.csv')
    #input_path = input_path.replace ("/","//")
    df = pd.read_csv(file)
    df = df.dropna()
    df['processed'] = df[label].apply(lambda x:nltk.word_tokenize(x))
    df['processed'] = df['processed'].apply(remove_punct_numbers)
    df['processed'] = df['processed'].apply(remove_stopwords)
    df['processed'] = df['processed'].apply(lematize)
    return df

df = input_func(train_data, 'text_ocr')
df.to_csv('train_data_processed.csv')

df = input_func(test_data, 'OCR_extracted_text')
df.to_csv('test_data_processed.csv')

# TASK1 using LR/NB

In [ ]:
#reading csv file
def read_corpus(file):
  return pd.read_csv(file)
  
train_data = '/content/gdrive/MyDrive/NLP_Project/train_data_processed.csv'
test_data = '/content/gdrive/MyDrive/NLP_Project/test_data_processed.csv'
test_true = '/content/gdrive/MyDrive/NLP_Project/Test_Actual_Final.csv'


In [ ]:
#Dataset containing true values 

true_df = read_corpus(test_true)
#Extracting the first digit (1, 0 , -1) from Labels 

true_df['Sentiment'] = true_df['Labels'].str.split('_').str[0]
true_df['Sentiment'] = true_df['Sentiment'].astype(int)
true_df.head()

,Unnamed: 0,Image_name,Labels,Sentiment
0,0,chuck_chuck_norris_meme_10.jpg,1_1100_1100,1
1,1,dr_evil_NDBB96K.png,1_0100_0200,1
2,2,misog_2109e457d636565e2e06dce39874c5231e1.jpg,1_1110_1120,1
3,3,obama_2691536739_469698809820026_263513986_n.jpg,0_1111_1121,0
4,4,kim_threat-kim-jong-un-allegedly-working-on-mu...,0_0000_0000,0


In [ ]:
#Dataset containing the Train data
train_df = read_corpus(train_data)

train_df.head()

,Unnamed: 0,Unnamed: 0.1,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,processed
0,0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive,"['look', 'friend', 'lightyear', 'sohalikut', '..."
1,1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive,"['best', 'yearchallenge', 'complete', 'less', ..."
2,2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,"['sam', 'thorne', 'strippin', 'follow', 'follo..."
3,3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,"['year', 'challenge', 'sweet', 'dee', 'edition']"
4,4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,"['year', 'challenge', 'filter', 'hilarious', '..."


In [ ]:
train_df['overall_sentiment'].value_counts()


positive         3057
neutral          2157
very_positive    1001
negative          469
very_negative     146
Name: overall_sentiment, dtype: int64

In [ ]:
#converting categorical to numerical 

train_df['overall_sentiment'].replace({'very_negative': -1, 'negative': -1, 'neutral': 0, 'positive': 1, 'very_positive': 1}, inplace=True)
train_df['overall_sentiment'].value_counts()

 1    4058
 0    2157
-1     615
Name: overall_sentiment, dtype: int64

In [ ]:
print(train_df.humour.value_counts())
print(train_df.sarcasm.value_counts())
print(train_df.offensive.value_counts())
print(train_df.motivational.value_counts())

funny         2394
very_funny    2176
not_funny     1618
hilarious      642
Name: humour, dtype: int64
general            3430
not_sarcastic      1516
twisted_meaning    1499
very_twisted        385
Name: sarcasm, dtype: int64
not_offensive        2657
slight               2536
very_offensive       1424
hateful_offensive     213
Name: offensive, dtype: int64
not_motivational    4421
motivational        2409
Name: motivational, dtype: int64


Task B:

Not humorous => 0 and Humorous (funny, very funny, hilarious) => 1
Not Sarcastic => 0 and Sarcastic (general, twisted meaning, very twisted) => 1
Not offensive => 0 and Offensive (slight, very offensive, hateful offensive) => 1
Not Motivational => 0 and Motivational => 1

In [ ]:

train_df.humour[train_df['humour']!='not_funny']= 1
train_df.humour[train_df['humour']=='not_funny']= 0
train_df.sarcasm[train_df['sarcasm']!='not_sarcastic']= 1
train_df.sarcasm[train_df['sarcasm']=='not_sarcastic']= 0
train_df.offensive[train_df['offensive']!='not_offensive']= 1
train_df.offensive[train_df['offensive']=='not_offensive']= 0
train_df.motivational[train_df['motivational']!='not_motivational']= 1
train_df.motivational[train_df['motivational']=='not_motivational']= 0

In [ ]:
train_df.head()

,Unnamed: 0,Unnamed: 0.1,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,processed
0,0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,1,1,0,0,1,"['look', 'friend', 'lightyear', 'sohalikut', '..."
1,1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,0,1,0,1,1,"['best', 'yearchallenge', 'complete', 'less', ..."
2,2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,1,0,0,0,1,"['sam', 'thorne', 'strippin', 'follow', 'follo..."
3,3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,1,1,1,1,1,"['year', 'challenge', 'sweet', 'dee', 'edition']"
4,4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,1,1,1,0,0,"['year', 'challenge', 'filter', 'hilarious', '..."


In [ ]:
print(train_df.humour.value_counts())
print(train_df.sarcasm.value_counts())
print(train_df.offensive.value_counts())
print(train_df.motivational.value_counts())

1    5212
0    1618
Name: humour, dtype: int64
1    5314
0    1516
Name: sarcasm, dtype: int64
1    4173
0    2657
Name: offensive, dtype: int64
0    4421
1    2409
Name: motivational, dtype: int64


In [ ]:
#Dataset containing the processed text of test data

test_df = read_corpus(test_data)
test_df.head()

,Unnamed: 0,Image_name,Image_URL,OCR_extracted_text,corrected_text,processed
0,0,chuck_chuck_norris_meme_10.jpg,https://gtmemes.com/wp-content/uploads/2019/03...,Some magicians can walk on water Chuck Norris...,Some magicians can walk on water Chuck Norris...,"['magician', 'walk', 'water', 'chuck', 'norris..."
1,1,dr_evil_NDBB96K.png,https://i.imgur.com/NDBB96K.png,ONE MILLION DOLLARS made on imgur,ONE MILLION DOLLARS made on imgur,"['one', 'million', 'dollar', 'make', 'imgur']"
2,2,misog_2109e457d636565e2e06dce39874c5231e1.jpg,https://media0ch-a.akamaihd.net/83/96/9e457d63...,Me: Mom can my friend sleep over? Mom: That's ...,Me: Mom can my friend sleep over? Mom: That's ...,"['mom', 'friend', 'sleep', 'mom', 'fine', 'boy..."
3,3,obama_2691536739_469698809820026_263513986_n.jpg,http://politicalmemes.com/wp-content/uploads/2...,THIS GUY INHERITED A MESS. DID HE WHINE ABOUT ...,THIS GUY INHERITED A MESS. DID HE WHINE ABOUT ...,"['guy', 'inherit', 'mess', 'whine', 'foxed', '..."
4,4,kim_threat-kim-jong-un-allegedly-working-on-mu...,https://pics.me.me/threat-kim-jong-un-allegedl...,THREAT: Kim Jong Un allegedly working on multi...,THREAT: Kim Jong Un allegedly working on multi...,"['threat', 'kim', 'jong', 'un', 'allegedly', '..."


In [ ]:
#required data

#for training
train_text = train_df['processed']  #X_train
sentiment = train_df['overall_sentiment'] #Y_train

#for testing
test_text = test_df['processed'] #X_test

#for predicting
actual_test_sentiment = true_df['Sentiment'] #Y_test

In [ ]:
#1. Count vectorizer for bag of words

cv = CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
cv_train = cv.fit_transform(train_text)
cv_test = cv.transform(test_text)



BOW train: (6830, 68600)
BOW test: (1859, 68600)


In [ ]:
#2. using TFIDF

vectorizer = TfidfVectorizer(ngram_range=(1, 3))

cv_train = vectorizer.fit_transform(train_text)    
cv_test = vectorizer.transform(test_text)


In [ ]:
#training the model using LR
model = LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=42)

#model = MultinomialNB()

#Fitting the model for Bag of words
model.fit(cv_train, sentiment)

print("\n Training Accuracy: ", model.score(cv_train, sentiment)*100)



 Training Accuracy:  95.2415812591508


In [ ]:
#making prediction
predicted_test_sentiment = model.predict(cv_test)

#calculating accuracy

print(' Test accuracy is {}'.format(accuracy_score(actual_test_sentiment, predicted_test_sentiment) * 100))
print(" F1 Score: {:.2f}".format(f1_score(actual_test_sentiment, predicted_test_sentiment, average='macro') * 100))
print(" Precision Score: {:.2f}".format(precision_score(actual_test_sentiment, predicted_test_sentiment, average='macro') * 100))
print(" Recall Score: {:.2f}".format(recall_score(actual_test_sentiment, predicted_test_sentiment, average='macro') * 100))

report = classification_report(actual_test_sentiment, predicted_test_sentiment)
print(report)

 Test accuracy is 58.095750403442715
 F1 Score: 26.30
 Precision Score: 30.41
 Recall Score: 33.18
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       172
           0       0.32      0.03      0.06       586
           1       0.59      0.96      0.73      1101

    accuracy                           0.58      1859
   macro avg       0.30      0.33      0.26      1859
weighted avg       0.45      0.58      0.45      1859



# TASK2 using LR/NB

In [ ]:
## Import The Dataset  
train_df=pd.read_csv(train_data)
test_df,true_df=pd.read_csv(test_data),pd.read_csv(test_true)

train_df.humour[train_df['humour']!='not_funny']='1'
train_df.humour[train_df['humour']=='not_funny']='0'
train_df.sarcasm[train_df['sarcasm']!='not_sarcastic']='1'
train_df.sarcasm[train_df['sarcasm']=='not_sarcastic']='0'
train_df.offensive[train_df['offensive']!='not_offensive']='1'
train_df.offensive[train_df['offensive']=='not_offensive']='0'
train_df.motivational[train_df['motivational']!='not_motivational']='1'
train_df.motivational[train_df['motivational']=='not_motivational']='0'

In [ ]:
def Multilabel(classifier,train_df,test_df,true_df):
    X_train=train_df['processed']
    X_test=test_df['processed']
    
    #### Create Bag of words 
    cv = CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
    cv_train = cv.fit_transform(X_train)
    cv_test = cv.transform(X_test)
    
    if classifier.lower()=='lr':
        # Using pipeline for applying logistic regression and one vs rest classifier
        model_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
    elif classifier.lower()=='nb':
        # Using pipeline for applying Naive Bayes and one vs rest classifier
        model_pipeline  = Pipeline([
                ('clf', OneVsRestClassifier(MultinomialNB(), n_jobs=-1)),
            ])
        
    
    categories={'humour':0,'sarcasm':1,'offensive':2,'motivational':3}
    

    for category in categories.keys():
        print('*Processing {} category...*'.format(category))
    
        # Training model pipeline on train data
        model_pipeline.fit(cv_train, train_df[category])
        
        # Applying k-Fold Cross Validation
        accuracies = cross_val_score(model_pipeline, X = cv_train, y = train_df[category], cv=KFold(n_splits=5))
        print("\n Accuracy: {:.2f} %".format(accuracies.mean()*100))
        print(" Standard Deviation: {:.2f} %".format(accuracies.std()*100))
        
        # calculating test accuracy
        predicted = model_pipeline.predict(cv_test)
        y_test=true_df['Labels'].str.split('_').str[1].str[categories[category]]
        report = classification_report(y_test, predicted)
        
        print(report)
        print(' Test accuracy is {}'.format(accuracy_score(y_test,predicted)))
        print(" F1 Score: {:.2f}".format(f1_score(y_test, predicted, average='macro') * 100))
        print(" Precision Score: {:.2f}".format(precision_score(y_test, predicted, average='weighted') * 100))
        print(" Recall Score: {:.2f}".format(recall_score(y_test, predicted, average='weighted') * 100))
        print("\n")

In [ ]:
Multilabel('LR',train_df,test_df,true_df)

*Processing humour category...*

 Accuracy: 76.31 %
 Standard Deviation: 1.28 %
              precision    recall  f1-score   support

           0       0.18      0.00      0.01       443
           1       0.76      0.99      0.86      1416

    accuracy                           0.76      1859
   macro avg       0.47      0.50      0.44      1859
weighted avg       0.62      0.76      0.66      1859

 Test accuracy is 0.7579343733189887
 F1 Score: 43.55
 Precision Score: 62.33
 Recall Score: 75.79


*Processing sarcasm category...*

 Accuracy: 77.80 %
 Standard Deviation: 0.87 %
              precision    recall  f1-score   support

           0       0.11      0.00      0.00       418
           1       0.77      0.99      0.87      1441

    accuracy                           0.77      1859
   macro avg       0.44      0.50      0.44      1859
weighted avg       0.63      0.77      0.68      1859

 Test accuracy is 0.771382463690156
 F1 Score: 43.78
 Precision Score: 62.54
 Recall

In [ ]:
Multilabel('NB',train_df,test_df,true_df)

*Processing humour category...*

 Accuracy: 39.55 %
 Standard Deviation: 0.82 %
              precision    recall  f1-score   support

           0       0.24      0.06      0.10       443
           1       0.76      0.94      0.84      1416

    accuracy                           0.73      1859
   macro avg       0.50      0.50      0.47      1859
weighted avg       0.64      0.73      0.66      1859

 Test accuracy is 0.7310381925766541
 F1 Score: 66.46
 Precision Score: 63.84
 Recall Score: 73.10


*Processing sarcasm category...*

 Accuracy: 36.08 %
 Standard Deviation: 0.78 %
              precision    recall  f1-score   support

           0       0.25      0.06      0.09       418
           1       0.78      0.95      0.86      1441

    accuracy                           0.75      1859
   macro avg       0.51      0.50      0.47      1859
weighted avg       0.66      0.75      0.68      1859

 Test accuracy is 0.750403442711135
 F1 Score: 68.33
 Precision Score: 65.81
 Recall